# Gradient verification for the 1D diffusion case with 1st order kinetics

The goal here is to verify the correctness of the adjoint state method in HYTEC by comparing the results with the demonstrator's which are considered as correct.

In [ ]:
import copy
from pathlib import Path
from typing import Dict

import gstools as gs
import matplotlib.pyplot as plt
import nested_grid_plotter as ngp
import numpy as np
import pandas as pd
import pyrtid
from IPython.display import HTML
from matplotlib.animation import HTMLWriter

gs.config.USE_RUST = True  # Use the rust implementation of gstools


- Check package/software versions

In [ ]:
pyrtid.utils.show_versions()

- Create a directory to store the exported figures

In [ ]:
ipynb_path = os.path.dirname(os.path.realpath("__file__"))
fig_save_path = Path(ipynb_path, "exported_figures")
fig_save_path.mkdir(parents=True, exist_ok=True)  # make sure that the directory exists

- Define some configurations for the plots

In [ ]:
# Some configs for the plots
new_rc_params = {
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica", "DejaVu Sans"],
    "font.size": 16,
    "text.usetex": False,
    "savefig.format": "svg",
    "svg.fonttype": "none",  # to store text as text, not as path
    "savefig.facecolor": "w",
    "savefig.edgecolor": "k",
    "savefig.dpi": 300,
    "figure.constrained_layout.use": True,
    "figure.facecolor": "w",
    # "axes.facecolor": "w",
}
csfont = {"fontname": "Comic Sans MS"}
hfont = {"fontname": "Helvetica"}
plt.rcParams.update(new_rc_params)

- Define a runner

In [ ]:
if runner_type == RunnerType.FRONTAL:
    runner = FrontalHytecRunner(
        hytec_binary_path_or_alias=hytec_binary_path_or_alias,
        mpi_binary_path_or_alias=mpi_binary_path_or_alias,
        nb_cpu=4,
        freq_checks_is_simu_over_sec=5,
    )
elif runner_type == RunnerType.SLURM:
    job_config = JobConfig(
        hytec_binary_path_or_alias=hytec_binary_path_or_alias,
        nb_nodes=1,
        ncpus=4,
        queue="geo-cpu",
        mpi_binary_path_or_alias=mpi_binary_path_or_alias,
        dos2unix_binary_path_or_alias=dos2unix_binary_path_or_alias,
    )
    runner = SlurmHytecRunner(
        job_config=job_config,
        freq_checks_is_simu_over_sec=5,
    )
elif runner_type == RunnerType.QSUB:
    job_config = JobConfig(
        hytec_binary_path_or_alias=hytec_binary_path_or_alias,
        nb_nodes=1,
        ncpus=4,
        queue="Omines_cpu",
        mpi_binary_path_or_alias=mpi_binary_path_or_alias,
        dos2unix_binary_path_or_alias=dos2unix_binary_path_or_alias,
    )
    runner = QsubHytecRunner(
        job_config=job_config,
        freq_checks_is_simu_over_sec=5,
    )
else:
    raise Exception("Could not created runner")

## Forward problem

- Define a very simple pure diffusion - advection case in 1D.

In [ ]:
nx = 14  # number of voxels along the x axis
ny = 7  # number of voxels along the y axis
dx = 10.0  # voxel dimension along the x axis
dy = 10.0  # voxel dimension along the y axis
nt = 300  # number of time steps
dt = 43200.0  # timestep in seconds
c0 = 0  # general initial cocentration
# Hydro parameters
D0 = 1e-10  # general initial diffusion coefficient [m2/s]
k0 = 1e-4  # general permeability
w0 = 0.23  # general porosity [fraction]
wdiff = 1.0  # Diffusion
wadv = 1.0  # Advection
# Chemistry parameters
c0 = 0.0  # general initial concentration [molal]
M0 = 0.001  # mineral grade [mol/kg] -> kg of water
kv = -6.9e-9  # kinetic rate,       [mol/m2/s]
moleweight = 270.0  # molar weight [g/mol]
surface = 500  # cm2/g
As = moleweight * surface / 1e4  # specific area [m2/mol]
logK = 3.2
Ks = 1.0 / pow(10, logK)  # solubility constant [no unit]
wmin = 1.0  # Mineral dissolution

# Values for the x axis ()meters
x_positions_in_meters = np.arange(nx) * dx + dx / 2

- Five observation wells

In [ ]:
injection_locations = [(3, 3)]
production_locations = [(10, 3)]

The mineral grades are defined in [mol/kg]. We calculate the conversion factor to obtain ppm and perform easier mass balances.

ConvU: parameter for converting the mineral content in [mol/kg] to metal grade in [ppm]. Note that the ConvU parameter is specific to the Uranium carrier phase: here Uraninite.

$C_{Uraninite}[\frac{mol}{kg}] = convU \times T_{Uranium}[ppm]$


and


$conv_u = \frac{1.023 \times density_{rock}}{238*porosity_{rock} \times 1000}$

Note the 1.023 is the conversion factor from molar mass to molal mass in CHESS.

In [ ]:
conv_u: float = 1.023 * 1.63 / (238.0 * w0 * 1000)
print(f"conv_u = {conv_u:.4e}")

- Create an initial gaussian spatial distribution for the mineral

In [ ]:
# Create a Gaussian Covariance Model just for the example
# To vary the results, change the seed :)
seed = 6

min_val = 10 * conv_u  # 10 ppm
max_val = 500 * conv_u  # 500 ppm
# Compute the mean and the standard deviation that the distribution should have so that
# <99% of the values are between min and max ~ 6 sigmas
mean = (max_val + min_val) / 2
stdev = (max_val - min_val) / 3 / 2  # std ~ 1/6 of the distribution interval
len_scale = 2

s_init_true = gen_random_ensemble(
    model=gs.covmodel.Gaussian,
    n_ensemble=1,
    var=stdev**2,
    len_scale=len_scale,
    mean=mean,
    nx=nx,
    ny=ny,
    seed=seed,
)[0, :, :, 0]


# Initial estimate = an homogeneous value
s_init_estimate = np.ones((nx, ny)) * 50 * conv_u  # 50 ppm

plotter = FieldPlotter(
    fig_params={"constrained_layout": True, "figsize": (12, 6)},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1", "ax1-2"]], sharey=True, sharex=True)
    },
)
plotter.plot_2d_field(
    ax_names=["ax1-1", "ax1-2"],
    data={
        "True": s_init_true / conv_u,
        "Estimated": s_init_estimate / conv_u,
    },
    cbar_title="Metal grade $[ppm]$",
    imshow_kwargs={"cmap": plt.get_cmap("jet"), "extent": [0.0, nx * dx, 0.0, ny * dy]},
    xlabel="X [m]",
    ylabel="Y [m]",
)
plotter.subfigs["fig0"].suptitle("Metal grade [ppm]", fontweight="bold")

fname = "Min_T_Cinet_field_true_vs_initial_estimation"
for format in ["png", "pdf"]:
    plotter.savefig((fig_save_path.joinpath(f"{fname}.{format}")), format=format)

- Note: The initial concentration is null.

- Create the flowrates for the wells: 12 m3/h decreasing following an exponential... for the producers with a balanced injection (2 m3/h per associated cell for the injectors).

In [ ]:
def gen_flowrates(amplitude: float, coef: float, nt: int, dt: float) -> np.ndarray:
    """Generate flowrates with a given amplitude and decrease coefficient.

    Parameters
    ----------
    amplitude : float
        Amplitude in m3/h.
    coef : float
        Decrease coefficient.
    nt : int
        Number of timesteps
    dt : float
        Timesteps.

    Returns
    -------
    np.ndarray
        The flowrates.
    """
    return amplitude * np.exp(coef * np.arange(nt) * dt)

- Generate a flowrates with an initial amplitude at 1 m3/h and plot it

In [ ]:
flowrates = gen_flowrates(1.0, 0, nt, dt=1.0)  # m/h

plt.plot(np.arange(nt), flowrates)
plt.ylim(0.0, 1.10)
plt.xlabel("time [d]")

### Forward problem in demonstrator

In [ ]:
# We define a function to easily generate a model.
def create_base_model() -> RTModel:
    return RTModel(
        nx,
        ny,
        dx,
        dy,
        nt,
        dt,
        c0,
        D0,
        k0,
        w0,
        M0,
        kv,
        As,
        Ks,
        wdiff=wdiff,
        wadv=wadv,
        wmin=wmin,
    )

### Forward problem definition in HYTEC

In [ ]:
# Create an empty simulation in a non existing folder
simu_base = HytecSimulation("simu_base", Path.cwd().joinpath("simu_base"))

htc = simu_base.handlers.htc
htc_path = simu_base.htc_file_path

# Add a TDB file
simu_base.link_tdb("./../../../../TDB/chess.tdb")  # This is relative to the htc file

htc.output_format = "vtk"

# Hydrodynamic parameters definition
htc.flow_regime = "stationary"
htc.porosity = w0
htc.permeability = k0
htc.permeability_units = "m/s"
htc.diffusion_coefficient = D0
htc.diffusion_coefficient_units = "m2/s"
htc.head = 0

# Boundary definition
htc.grid_regime = "rectangle"
htc.domain = f"{nx*dx},{nx} {ny*dy},{ny}"

# Geometry
dzone = HytecZone("domain", htc_path)
dzone.geometries = ["domain"]
dzone.geochem = "chem_base"
htc.zones_dict["domain"] = dzone

# Boundary conditions
left_border = HytecBoundary("border_left", htc_path)
left_border.coordinates = f"0,0, 0,{ny*dy} m"
htc.boundaries_dict["left"] = left_border

right_border = HytecBoundary("border_right", htc_path)
right_border.coordinates = f"{nx*dx},{ny*dy}, {nx*dx},0 m"
left_border.flow_condition = "constant-head at 0 m"
htc.boundaries_dict["right"] = right_border

top_border = HytecBoundary("border_top", htc_path)
top_border.coordinates = f"0,{ny*dy}, {nx*dx},{ny*dy} m"
htc.boundaries_dict["top"] = top_border

bottom_border = HytecBoundary("border_bottom", htc_path)
bottom_border.coordinates = f"0,0, {nx*dx},0 m"
htc.boundaries_dict["bottom"] = bottom_border

for border in htc.boundaries_dict.values():
    border.flow_condition = "constant-head at 0 m"

# Geochemical unit definition
htc.report = "full"
htc.redox = "disabled"
new_unit = HytecGeochemUnit("chem_base")
new_unit.concentrations["T_Cinet"] = Concentration("T_Cinet", c0, "molal")
new_unit.minerals["Min_T_Cinet"] = Mineral(
    "Min_T_Cinet", M0, grade_units="mol/kg", surface=As, surface_units="m2/mol"
)
htc.geochem_units_dict[new_unit.name] = new_unit

new_unit = HytecGeochemUnit("injected_solution")
new_unit.concentrations["T_Cinet"] = Concentration("T_Cinet", c0, "molal")
new_unit.minerals["Min_T_Cinet"] = Mineral(
    "Min_T_Cinet", 0.0, grade_units="mol/kg", surface=As, surface_units="m2/mol"
)
htc.geochem_units_dict[new_unit.name] = new_unit


# Time specification
htc.duration = nt * dt
htc.duration_units = "s"
htc.timestep = (
    f"variable {{\n\tstart = {dt} s\n\tmaximum = {dt} s\n\tcourant-factor = 20\n}}"
)
htc.samples = nt

# define, extend

htc.define_list.append(
    HytecDefinition(
        phase="basis",
        species="T_Cinet",
        content=f"""\n\tmoleweight = {moleweight} g/mol\n""",
    )
)

htc.define_list.append(
    HytecDefinition(
        phase="mineral",
        species="Min_T_Cinet",
        content=f"""
	composition = 1 T_Cinet
	logK = {logK}
	surface = {surface} cm2/g
	kinetics {{
		rate = {kv} mol/m2/s
		area = Min_T_Cinet
		y-term, species = Min_T_Cinet
	}}\n""",
    )
)

# select
htc.verbose = "enabled"
htc.grid_select_list = [
    "time in s",
    "node-number",
    "x-flowrate in m/s",
    "permeability in m/s",
    "head in m",
    "porosity",
    "Min_T_Cinet in mol/kg",
    "T_Cinet in mol/kg",
]

# exclude
htc.exclude_list = ["minerals", "colloids", "gases"]

- Create the injection/pumping file: 1 column per well + 1 column for the time (days)

In [ ]:
src_term_path = Path.cwd().joinpath("source_terms.dat")
src_term_data: NDArrayFloat = np.zeros(
    (nt, len(injection_locations) + len(production_locations) + 1)
)
src_term_data[:, 0] = np.arange(stop=nt + 1, start=1)

- Create one zone per well with the correct flowrates: negative for producers and positive for injectors.

In [ ]:
prod_flw_coef = -2  # m3/h

# 1) Add the producer wells
count_prod: int = 0  # to avoid unbounded values in the next loop
for count_prod, (ix, iy) in enumerate(production_locations):
    name = f"producer_{ix}_{iy}"
    zone = HytecZone(name, htc_path=simu_base.htc_file_path)
    zone.geometries.append(f"rectangle {(ix + 0.5) * dx},{(iy + 0.5) * dy}, {dx},{dy}")
    zone.geochem = "chem_base"
    zone.global_flux = name  # TODO: need to handle the type of obs with the selects
    zone.source = f"{prod_flw_coef * flowrates[0]} m3/h"  # using chem_A ...
    zone.modify_list.append(
        f"at $1 d, source = ${count_prod+2} m3/h using chem_base from "
    )
    zone.modify_src_files.append(src_term_path)
    simu_base.handlers.htc.zones_dict[name] = zone

    # Update the src_terms file -> 2 m3/h for a producer
    src_term_data[:, count_prod + 1] = flowrates * prod_flw_coef

n_prod_linked_list = [1]
inj_flw_coef = -len(production_locations) * prod_flw_coef / sum(n_prod_linked_list)

# 2) Add the injector wells
for count_inj, (ix, iy) in enumerate(injection_locations):
    name = f"injector_{ix}_{iy}"
    zone = HytecZone(name, htc_path=simu_base.htc_file_path)
    zone.geometries.append(f"rectangle {(ix + 0.5) * dx},{(iy + 0.5) * dy}, {dx},{dy}")
    zone.geochem = "chem_base"
    zone.global_flux = name  # TODO: need to handle the type of obs with the selects
    zone.source = f"{n_prod_linked_list[count_inj] * inj_flw_coef * flowrates[0]} m3/h using injected_solution"
    zone.modify_list.append(
        f"at $1 d, source = ${count_prod + count_inj +3} m3/h using chem_base from "
    )
    zone.modify_src_files.append(src_term_path)
    simu_base.handlers.htc.zones_dict[name] = zone

    src_term_data[:, count_prod + count_inj + 2] = (
        flowrates * inj_flw_coef * n_prod_linked_list[count_inj]
    )

# 3) Add the src_data_file (need to write and read the file... which is a bit stupid...)
np.savetxt(src_term_path, src_term_data, fmt="%.3f", delimiter="\t")
htc.modify_src_file_dict[src_term_path] = open(src_term_path, "r").read()

- Create two models from this base simulation

In [ ]:
simu_true = copy.deepcopy(simu_base)
simu_true.update_root_and_name(new_root="simu_true", new_name="simu_true")
simu_estimate = copy.deepcopy(simu_base)
simu_estimate.update_root_and_name(new_root="simu_estimate", new_name="simu_estimate")

- Add the mineral fields to the simulations

In [ ]:
index: NDArrayFloat = np.arange(nx * ny)

# True
data_true = pd.DataFrame(
    data={
        "node-number": index,
        # x and y are cell centers
        # "x": (index % nx) * dx + dx / 2,
        "Min_T_Cinet": s_init_true.ravel(),
        "Min_T_Cinet2": s_init_true.ravel(),
    },  # need to flatten the parameter
    index=index,
)
simu_true.add_param_file_data(ParameterFiles.MINERALS, data_true)

# Estimated
data_estimated = pd.DataFrame(
    data={
        "node-number": index,
        # x and y are cell centers
        # "x": (index % nx) * dx + dx / 2,
        "Min_T_Cinet": s_init_estimate.ravel(),
        "Min_T_Cinet2": s_init_estimate.ravel(),
    },  # need to flatten the parameter
    index=index,
)
simu_estimate.add_param_file_data(ParameterFiles.MINERALS, data_estimated)

- Checking the hydro parameters have been correctly set-up

In [ ]:
simu_true.handlers.minerals.data

In [ ]:
simu_estimate.handlers.minerals.data

In [ ]:
simu_true.write_input_files()
simu_estimate.write_input_files()
runner.run(simu_true)
runner.run(simu_estimate)

### Comparison between hytec and the demonstrator for the forward problem

- Reading the results for the true diffusion simulation

In [ ]:
simu_true.read_hytec_results()
simu_true.handlers.results.grid_res_columns

In [ ]:
fwd_conc_true_hytec = simu_true.handlers.results.extract_field_from_grid_res(
    field="T_Cinet [mol/kg]", nx=nx, ny=ny
)
fwd_min_true_hytec = simu_true.handlers.results.extract_field_from_grid_res(
    field="Min_T_Cinet [mol/kg]", nx=nx, ny=ny
)
# Getting sample time. The unit is the same than the one defined for the simulation duration in the htc
flux_sample_times = simu_true.handlers.results.get_sample_times_from_grid_res()

- Reading the results for the estimated diffusion simulation

In [ ]:
simu_estimate.read_hytec_results()
simu_estimate.handlers.results.grid_res_columns

In [ ]:
# Get the results on a 3D grid with the last dimension as time step
fwd_conc_estimate_hytec = simu_estimate.handlers.results.extract_field_from_grid_res(
    field="T_Cinet [mol/kg]",
    nx=nx,
    ny=ny,
)
fwd_min_estimate_hytec = simu_estimate.handlers.results.extract_field_from_grid_res(
    field="Min_T_Cinet [mol/kg]", nx=nx, ny=ny
)

- Top view of the adjoint concentrations

- Adjoint variable at the producer locations

In [ ]:
plotter = StateVariableSlicePlotter(nb_obs_loc=len(production_locations), nb_obs_type=1)
for i, (ix, iy) in enumerate(production_locations):
    x = (ix + 0.5) * dx
    y = (iy + 0.5) * dy
    obs_well_name = f"obs. well @ node #{ix}_{iy}"

    plotter.plot_state(
        row_id=i,
        obs_well_name=obs_well_name,
        data={
            "Tracer": {
                # "True demonstrator": (
                #     solver_true.model.fwd_conc[node_id, 0, :] * 1e3,
                #     {"c": "b", "linestyle": "--", "zorder": 3},
                # ),
                # "A priori (estimate) model demonstrator": (
                #     solver_estimate.model.fwd_conc[node_id, 0, :] * 1e3,
                #     {"c": "r", "linestyle": "--", "zorder": 3},
                # ),
                "True model hytec": (
                    fwd_conc_true_hytec[ix, iy, 0, :] * 1e3,
                    {"c": "g"},
                ),
                "A priori (estimate) model hytec": (
                    fwd_conc_estimate_hytec[ix, iy, 0, :] * 1e3,
                    {"c": "k"},
                ),
            },
        },
    )
plotter.add_fig_legend(ncol=2)

fname = "tracer_concentration_true_vs_estimate_at_obs_1D"
for format in ["png", "pdf"]:
    plotter.savefig((fig_save_path.joinpath(f"{fname}.{format}")), format=format)

In [ ]:
plotter = ngp.AnimatedPlotter(
    fig_params={"constrained_layout": True, "figsize": (12, 10)},
    subfigs_params={"nrows": 2},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1", "ax1-2"]], sharey=True, sharex=True),
        "fig1": dict(mosaic=[["ax2-1", "ax2-2"]], sharey=True, sharex=True),
    },
)

# Static plot
for ax_name in ["ax1-1", "ax1-2"]:
    plotter.plot_1d_static(
        ax_name=ax_name,
        data={"Initial": {"data": c_ * 1e3, "kwargs": {"c": "red"}}},
    )

for ax_name in ["ax2-1", "ax2-2"]:
    plotter.plot_1d_static(
        ax_name=ax_name,
        data={
            "Initial": {
                "data": fwd_min_true_hytec[:, 1, 0, 0] / conv_u,
                "kwargs": {"c": "red"},
            }
        },
    )

# Animated plot
nb_frames = 30

plotter.animated_multi_plot(
    ax_name="ax1-1",
    nb_frames=nb_frames,
    data={
        "True": {
            "data": fwd_conc_true_hytec[:, 1, 0, :] * 1e3,
            "kwargs": {"c": "blue"},
        },
        "Estimated": {
            "data": fwd_conc_estimate_hytec[:, 1, 0, :] * 1e3,
            "kwargs": {"c": "green", "linestyle": "--"},
        },
    },
    title="HYTEC conc",
    xlabel="Node #",
    ylabel="[mmol/kg]",
)

# plotter.animated_multi_plot(
#     ax_name="ax1-2",
#     nb_frames=nb_frames,
#     data={
#         "True": {
#             "data": solver_true.model.fwd_conc[:, 0, :] * 1e3,
#             "kwargs": {"c": "blue"},
#         },
#         "Estimated": {
#             "data": solver_estimate.model.fwd_conc[:, 0, :] * 1e3,
#             "kwargs": {"c": "green", "linestyle": "--"},
#         },
#     },
#     title="Demonstrator conc",
#     xlabel="Node #",
# )

plotter.animated_multi_plot(
    ax_name="ax2-1",
    nb_frames=nb_frames,
    data={
        "True": {
            "data": fwd_min_true_hytec[:, 1, 0, :] / conv_u,
            "kwargs": {"c": "blue"},
        },
        "Estimated": {
            "data": fwd_min_estimate_hytec[:, 1, 0, :] / conv_u,
            "kwargs": {"c": "green", "linestyle": "--"},
        },
    },
    title="HYTEC conc",
    xlabel="Node #",
    ylabel="[ppm]",
)

# plotter.animated_multi_plot(
#     ax_name="ax2-2",
#     nb_frames=nb_frames,
#     data={
#         "True": {
#             "data": solver_true.model.fwd_min[:, 0, :] / conv_u,
#             "kwargs": {"c": "blue"},
#         },
#         "Estimated": {
#             "data": solver_estimate.model.fwd_min[:, 0, :] / conv_u,
#             "kwargs": {"c": "green", "linestyle": "--"},
#         },
#     },
#     title="Demonstrator min",
#     xlabel="Node #",
# )

for ax_name in ["ax1-1", "ax1-2", "ax2-1", "ax2-2"]:
    # Add some vertical lines to indicate the well
    for well_pos in production_locations:
        plotter.ax_dict[ax_name].plot(
            well_pos,
            0.0002,
            label="obs wells",
            marker="^",
            markersize=10,
            c="r",
            linestyle="none",
        )

    plotter.add_axis_legend(ax_name)

plotter.ax_dict["ax2-1"].set_ylim(0, plotter.ax_dict["ax2-1"].get_ylim()[1])

plotter.animate(nb_frames=nb_frames)
# Save the animation locally on the computer
fname_html = fig_save_path.joinpath("true_vs_estimate_conc_animation.html")
writer = HTMLWriter(fps=5, embed_frames=True)
writer.frame_format = "svg"  # Ensure svg format
plotter.animation.save(fname_html, writer=writer)

# Extract the svg from the html file (for animation in Latex)
ngp.extract_frames_from_embedded_html_animation(fname_html)

# Display the animation
HTML(fname_html.read_text())

- Plot the heads

In [ ]:
fwd_heads_true_hytec = simu_true.handlers.results.extract_field_from_grid_res(
    field="head [m]", nx=nx
)

plt.plot(fwd_heads_true_hytec[:, 0, 0, 0], label="initial")
plt.plot(fwd_heads_true_hytec[:, 0, 0, -1], label="final")
plt.ylabel("Head [m]")
plt.legend()

plt.savefig(str(fig_save_path.joinpath("Heads in forward.png")))

- Plot the speed

In [ ]:
fwd_darcy_true_hytec = simu_true.handlers.results.extract_field_from_grid_res(
    field="flowrate [m/s]", nx=nx
)

plt.plot(fwd_darcy_true_hytec[:, 0, 0, 0], label="initial")
plt.plot(fwd_darcy_true_hytec[:, 0, 0, -1], label="final")
plt.ylabel("x-flowrate [m/s]")
plt.legend()

plt.savefig(str(fig_save_path.joinpath("Flowrates in forward.png")))

## Inversion

- Create a function to add some noise to the observations

In [ ]:
noise_std = 1  # 5e-6  # This is an absolute value
rng = np.random.default_rng(2021)


def make_noisy(x: NDArrayFloat) -> NDArrayFloat:
    """Return the input with some added white noise.

    Note
    ----
    The parameters are hardcoded to be consistent in the notebook.
    Change the function directly.
    """
    mean_noise = 0.0  # mean
    return x
    return x + rng.normal(mean_noise, noise_std, x.shape)

- Check the intensity of the noise

In [ ]:
plt.figure(facecolor="w")
plt.plot(fwd_conc_true_hytec[node_id, 0, 0, :])
plt.plot(
    make_noisy(fwd_conc_true_hytec[node_id, 0, 0, :]), marker=".", linestyle="none"
)

### Demonstrator


In [ ]:
# Create an executor
executor = InversionExecutor(model_estimate)


param = PyrtidAdjustableParameter(
    name=PyrtidParameterName.INITIAL_GRADE,
    lbounds=0.0,
    ubounds=600.0 * conv_u,
)


observables: Dict[int, PyrtidObservable] = {}
for node_id in production_locations:
    vals = make_noisy(solver_true.model.fwd_conc[node_id, 0, :])
    timesteps = np.arange(vals.shape[0])

    observables[node_id] = PyrtidObservable(
        state_variable="tracer",
        location=(slice(node_id, node_id + 1, 1), slice(None)),
        timesteps=timesteps,
        values=vals,
        uncertainties=noise_std,
    )

# Compute the gradient both by finite difference and adjoint method
is_grad_ok = executor.is_gradient_correct(
    param,
    list(observables.values()),
)
print("Is the gradient correct: ", is_grad_ok)

### HYTEC

In [ ]:
# 1) Copy the base simulation
simu_inverse = copy.deepcopy(simu_estimate)
simu_inverse.update_root_and_name(new_root="simu_inverse", new_name="simu_inverse")

# 2) Add the observation wells
for count, ix in enumerate(production_locations):
    x = (ix + 0.5) * model_true.dx
    name = f"producer_{ix}"
    vals = make_noisy(fwd_conc_true_hytec[ix, 0, 0, :])
    odata = pd.DataFrame(
        data={
            "time [s]": flux_sample_times,
            "node-number": indices_to_hytec_node_number(ix, nx=nx),
            "T_Cinet [mol/kg]": vals,
            "T_Cinet uncertainties [mol/kg]": noise_std,
        }
    )

    # Add the observables for the area
    obs = Observable(
        zone_name=zone.name,
        column_index=3,
        root_relative_path=f"observables/well{count + 1}_o.dat",
        root_path=simu_inverse.root,
        type=ObservationType.GRID,
        uncertainties_column_index=4,
    )
    obs.handler.update_data(odata)
    simu_inverse.handlers.htc.zones_dict[name].observables.append(obs)

# 3) Update the htc file with some options
simu_inverse.handlers.htc.optimization = "enabled"
simu_inverse.handlers.htc.optimization_solver = OptimizationSolverConfig(
    solver_name="lbfgsb", adjoint_state="enabled", fd_gradient_check="initial"
)
simu_inverse.handlers.htc.adjusted_parameters_dict = {
    "mineral Min_T_Cinet": AdjustedParameterConfig(
        name="mineral Min_T_Cinet", lbounds=0.0, ubounds=600.0 * conv_u
    ),
}
simu_inverse.handlers.htc.adjoint_sampling = (
    nt  # number of samples on the adjoint variables
)

# 4)Write the input files
simu_inverse.write_input_files()

runner.run(simu_inverse)

# 7) Read results
simu_inverse.read_hytec_results()  # This fails if not results have been written

In [ ]:
simu_inverse.read_hytec_results()

In [ ]:
inverse_heads_hytec = simu_inverse.handlers.results.extract_field_from_grid_res(
    field="head [m]", nx=nx
)

print(inverse_heads_hytec.shape)

plt.plot(inverse_heads_hytec[:, 0, 0, 0])
plt.plot(inverse_heads_hytec[:, 0, 0, 1])

- Extract the adjoint variables computed for the first gradient

In [ ]:
print(f"columns = {simu_inverse.handlers.results.optim_res.adjoint_var_columns}")

In [ ]:
adj_conc_hytec = simu_inverse.handlers.results.extract_field_from_adj_var_res(
    field="adjoint-variable{T_Cinet} [m]", nx=nx
)
adj_conc_hytec.shape

### Results comparison

- Compare objective functions

In [ ]:
print(
    "demonstrator 1st obj function = ",
    executor.rt_solver.inverse_model.loss_scaled_history[0],
)
print("hytec 1st obj function = ", simu_inverse.handlers.results.optim_res.obj_funs[0])

Because the mineral, tracer and associated observations has been doubled in the case of HYTEC (to test if all is working correctly), the objective function is expected twice higer in the case of HYTEC.

- The adjoint concentrations should be quite the same both in shape and intensity.

In [ ]:
plotter = ngp.AnimatedPlotter(
    fig_params={"constrained_layout": True, "figsize": (6, 5)},
    subfigs_params={"nrows": 2},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1"]], sharey=True, sharex=True),
    },
)

# Animated plot
nb_frames = 30

plotter.animated_multi_plot(
    ax_name="ax1-1",
    nb_frames=nb_frames,
    data={
        "Adj": {
            "data": adj_conc_hytec[:, 0, 0, ::-1],
            "kwargs": {"c": "blue"},
        },
    },
    title="HYTEC conc",
    xlabel="Node #",
    ylabel="[mmol/kg]",
)

for ax_name, ax in plotter.ax_dict.items():
    # Add some vertical lines to indicate the well
    for well_pos in production_locations:
        ax.plot(
            well_pos,
            0.0002,
            label="obs wells",
            marker="^",
            markersize=10,
            c="r",
            linestyle="none",
        )

    plotter.add_axis_legend(ax_name)

plotter.animate(nb_frames=nb_frames)
# Save the animation locally on the computer
fname_html = fig_save_path.joinpath("true_vs_estimate_conc_animation.html")
writer = HTMLWriter(fps=5, embed_frames=True)
writer.frame_format = "svg"  # Ensure svg format
plotter.animation.save(fname_html, writer=writer)

# Extract the svg from the html file (for animation in Latex)
ngp.extract_frames_from_embedded_html_animation(fname_html)

# Display the animation
HTML(fname_html.read_text())

- Check the adjoint variables at the observation well

In [ ]:
plotter = StateVariableSlicePlotter(nb_obs_loc=len(production_locations), nb_obs_type=1)
for i, node_id in enumerate(production_locations):
    x = (node_id + 0.5) * model_true.dx
    obs_well_name = f"obs. well @ node #{node_id} (x={x}m)"

    plotter.plot_state(
        row_id=i,
        obs_well_name=obs_well_name,
        data={
            "Tracer adjoint variable": {
                # "Adj. var. Demonstrator": (
                #     executor.rt_solver.inverse_model.adj_conc[node_id, 0, :],
                #     {"c": "g"},
                # ),
                "Adj. var. HYTEC": (adj_conc_hytec[node_id, 0, 0, :], {"c": "r"}),
            }
        },
    )
plotter.add_fig_legend(ncol=2)

fname = "adjoint_concentrations_dem_vs_hytec_1D"
for format in ["png", "pdf"]:
    plotter.savefig((fig_save_path.joinpath(f"{fname}.{format}")), format=format)

- Get the gradients by the adjoint method and finite differences

Finally, plot the gradients comparison.

In [ ]:
# Get the HYTEC  Adj gradient
hytec_fd_gradient = simu_inverse.handlers.results.optim_res.fd_gradients[0]
hytec_adjoint_gradient = simu_inverse.handlers.results.optim_res.adjoint_gradients[0]

- The demonstrator gradient must be dvided by two to see the equivalence, because the objective function is two times higher in HYTEC. Also, note that the gradient in the demonstrator has 40 values (40 grid cells), while it has 80 in HYTEC (40 grid cells * 2 minerals).

In [ ]:
# Here comes the python code
plotter = FieldPlotter()
plotter.plot_1d_field(
    ax_name="ax1-1",
    data={
        # "Adj. Demonstrator": {
        #     "data": param.grad_adj_history[0] / 2,
        #     "kwargs": {"c": "b"},
        # },
        # "FD Demonstrator": {
        #     "data": param.grad_fd_history[0] / 2,
        #     "kwargs": {"c": "r", "linestyle": "--"},
        # },
        "Adj. HYTEC": {"data": hytec_adjoint_gradient["value"], "kwargs": {"c": "g"}},
        "FD HYTEC": {
            "data": hytec_fd_gradient["value"],
            "kwargs": {"c": "k", "linestyle": "--"},
        },
    },
    title="Gradient at first iteration",
    xlabel="Node #",
)
plotter.add_fig_legend(ncol=2, bbox_y_shift=-0.1)

fname = "gradient_dem_vs_hytec_adj_vs_df_1d_diffusion"
for format in ["png", "pdf"]:
    plotter.savefig((fig_save_path.joinpath(f"{fname}.{format}")), format=format)

# Full optimization

In [ ]:
# 1) Copy the base simulation
simu_inverse2 = copy.deepcopy(simu_inverse)
simu_inverse2.update_root_and_name(
    new_root="simu_inverse_lbfgsb", new_name="simu_inverse_lbfgsb"
)

# 2) Update the htc file with some options
simu_inverse2.handlers.htc.optimization = "enabled"
simu_inverse2.handlers.htc.optimization_solver = OptimizationSolverConfig(
    solver_name="lbfgsb",
    max_number_iterations=4,
    max_number_fwd_model_eval=600,
    max_number_gradient_eval=15,
    max_number_hessian_eval=1,
    objfun_threshold=1e-1,
    objfun_min_change=0.0,
    param_min_change=0.0,
    gradient_min_norm=0.0,
    hessian_max_norm=0.0,
    grad_history_size=5,
    adjoint_state="enabled",
    fd_gradient_check="disabled",
)

# 4)Write the input files
simu_inverse2.write_input_files()
runner.run(simu_inverse2)

# 5) Read the results
simu_inverse2.read_hytec_results()

- Get the concentrations

In [ ]:
# Get the results on a 3D grid with the last dimension as time step
fwd_conc_inversed_hytec = simu_inverse2.handlers.results.extract_field_from_grid_res(
    field="T_Cinet [mol/kg]", nx=nx
)

- Get the gradients 

In [ ]:
print(simu_inverse2.handlers.results.optim_res.adjoint_gradients[0].columns)

adjoint_gradients = simu_inverse2.handlers.results.get_adjoint_gradient(
    "Min_T_Cinet", nx=nx, ny=ny
)
fd_gradients = simu_inverse2.handlers.results.get_fd_gradient(
    "Min_T_Cinet", nx=nx, ny=ny
)
adjusted_min_grade = simu_inverse2.handlers.results.get_adjusted_parameter_field(
    "Min_T_Cinet", nx=nx, ny=ny
)
obj_funs = simu_inverse2.handlers.results.optim_res.obj_funs

- Align gradients and objective functions: Because of the construction in HYTEC, unless I am mistaken, the objective function and fitted parameter vectors match exactly. On the other hand, there is an uncontrolled mismatch with the gradients due to the solver which, depending on its needs, may have to apply several objective functions in a row or several gradients. This function makes sure that the two match for a good rendering of the animations.

In [ ]:
grad_index = 0
grad_obj_funs = (
    simu_inverse2.handlers.results.optim_res.obj_funs_matching_adjoint_gradients
)

# Example: [3, 4, 4, 5, 7, 8, 8, 9]

aligned_adjoint_gradients = np.empty(
    (*adjoint_gradients[:, :, :, 0].shape, obj_funs.size)
)
aligned_adjoint_gradients.fill(np.nan)

previous_index: int = -1
for i, index in enumerate(grad_obj_funs):
    if index == previous_index:
        continue
    if index > obj_funs.size:
        continue
    previous_index = index
    aligned_adjoint_gradients[:, :, :, index - 1] = adjoint_gradients[:, :, :, i]

# Fill the first missing gradients
# Gradients 1 and 2 in the example
for i2 in range(grad_obj_funs[0] - 1):
    aligned_adjoint_gradients[:, :, :, i2] = aligned_adjoint_gradients[
        :, :, :, grad_obj_funs[0]
    ]

- Get the observations and the predictions vector to plot it against each others

In [ ]:
obs_vector = simu_inverse2.get_observation_vector()
pred_vector = simu_inverse2.get_results_matching_obs_vector(
    simu_inverse2.get_observables()
)

- Plot the main results

In [ ]:
plotter = ngp.AnimatedPlotter(
    fig_params={"figsize": (10.0, 8.0)}, subfigs_params={"nrows": 2, "ncols": 2}
)

# 1 frame per solver iteration
nb_frames: int = obj_funs.shape[0]

# 1) Gradient evolution
plotter.plot_1d_static(
    ax_name="ax1-1",
    data={
        "Initial": {"data": aligned_adjoint_gradients[:, 0, 0, 0], "kwargs": {"c": "b"}}
    },
    title="Gradient",
    xlabel="Node #",
)

plotter.animated_multi_plot(
    ax_name="ax1-1",
    nb_frames=nb_frames,
    data={
        "ADJ": {"data": aligned_adjoint_gradients[:, 0, 0, :], "kwargs": {"c": "g"}},
    },
)
plotter.ax_dict["ax1-1"].legend()


# 2) Parameter evolution
plotter.plot_1d_static(
    ax_name="ax1-2",
    data={
        "True": {"data": s_init_true / conv_u, "kwargs": {"c": "b"}},
        "Initial": {
            "data": adjusted_min_grade[:, 0, 0, 0] / conv_u,
            "kwargs": {"c": "orange"},
        },
    },
    title="Uraninite Field",
    xlabel="Node #",
    ylabel="[ppm]",
)

plotter.animated_multi_plot(
    ax_name="ax1-2",
    nb_frames=nb_frames,
    data={
        "Inverted": {
            "data": adjusted_min_grade[:, 0, 0, :] / conv_u,
            "kwargs": {"c": "r", "linestyle": "--"},
        },
    },
)
plotter.ax_dict["ax1-2"].legend()

# 3) Objective function
vals = obj_funs
obj_fun_vals = np.full((nb_frames, len(vals)), fill_value=np.nan)
for i in range(len(vals)):
    obj_fun_vals[i, : i + 1] = vals[: i + 1]

plotter.animated_multi_plot(
    ax_name="ax2-1",
    nb_frames=nb_frames,
    data={
        "Obj fun": {"data": obj_fun_vals.T, "kwargs": {"c": "r", "linestyle": "--"}},
    },
    title="Objective function",
    xlabel="Iteration #",
)
plotter.ax_dict["ax2-1"].set_yscale("log")

# 4) Observation vs predicted values
plot_observed_vs_simulated(
    plotter.ax_dict["ax2-2"],
    obs_vector=simu_inverse2.get_observation_vector() * 1000,
    pred_vector=simu_inverse2.get_results_matching_obs_vector(
        simu_inverse2.get_observables()
    )
    * 1000,
    pred_vector_initial=simu_estimate.get_results_matching_obs_vector(
        simu_inverse2.get_observables()
    )
    * 1000,
    unit="$mmol.l^{-1}$",
)

plotter.animate(nb_frames=nb_frames)
# Save the animation locally on the computer
fname_html = fig_save_path.joinpath("m_j_g_animation.html")
writer = HTMLWriter(fps=2, embed_frames=True)
writer.frame_format = "svg"  # Ensure svg format
plotter.animation.save(fname_html, writer=writer)

# Extract the svg from the html file (for animation in Latex)
ngp.extract_frames_from_embedded_html_animation(fname_html)

# Display the animation
HTML(fname_html.read_text())

- Plot the concentrations

In [ ]:
plotter = StateVariableSlicePlotter(nb_obs_loc=3, nb_obs_type=1)
for i, node_id in enumerate([6, 10, 15]):
    x = (node_id + 0.5) * model_true.dx
    obs_well_name = f"obs. well @ node #{node_id} (x={x}m)"

    plotter.plot_state(
        row_id=i,
        obs_well_name=obs_well_name,
        data={
            "Tracer (HYTEC)": {
                "True (noisy)": (
                    make_noisy(fwd_conc_true_hytec[node_id, 0, 0, :]),
                    {"c": "g", "linestyle": "none", "marker": ".", "alpha": 0.5},
                ),
                "A priori (estimate)": (
                    fwd_conc_estimate_hytec[node_id, 0, 0, :],
                    {"c": "r"},
                ),
                "Post inversion": (
                    fwd_conc_inversed_hytec[node_id, 0, 0, :],
                    {"c": "b"},
                ),
            },
        },
    )
plotter.add_fig_legend(ncol=2)

fname = "tracer_concentrations_after_inversion"
for format in ["png", "pdf"]:
    plotter.savefig((fig_save_path.joinpath(f"{fname}.{format}")), format=format)